# 3. Multi-class Classification and Neural Networks

- 작성자: [JWarmenhoven's Exercise 3](https://nbviewer.jupyter.org/github/JWarmenhoven/Coursera-Machine-Learning/blob/master/notebooks/Programming%20Exercise%203%20-%20Multi-class%20Classification%20and%20Neural%20Networks.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.io import loadmat
from scipy.optimize import minimize

from sklearn.linear_model import LogisticRegression

%matplotlib inline
import seaborn as sns
sns.set_context("notebook")
sns.set_style("white")

loading
- 내가 drive 연결을 못해서 colab까지 왔다..

In [4]:
data = loadmat("/content/drive/My Drive/study/ML(Andrew Ng)/data/ex3data1.mat")
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [5]:
weights = loadmat("/content/drive/My Drive/study/ML(Andrew Ng)/data/ex3weights.mat")
weights.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])

In [18]:
y = data['y']
X = np.c_[np.ones((data['X'].shape[0], 1)), data['X']]
print(f"X: {X.shape} (with intercept)", f"y: {y.shape}", sep = "\n")

X: (5000, 401) (with intercept)
y: (5000, 1)


In [19]:
theta1, theta2 = weights["Theta1"], weights["Theta2"]
print(f"theta1: {theta1.shape}", f"theta2: {theta2.shape}", sep = "\n")

theta1: (25, 401)
theta2: (10, 26)


In [20]:
sample = np.random.choice(X.shape[0], 20)
plt.imshow(X[sample, 1:].reshape(-1, 20).T)
plt.axis("off")
plt.show()

### Multiclass Classification

함수 정의와 분리하기엔 내용 측면이 너무 훅 날아가버리는 느낌이라 분리할 수 없었다.

In [21]:
def sigmoid(z):
  return(1/(1 + np.exp(-z)))

lr: logistic regression

In [22]:
def lrcostFunctionReg(theta, reg, X, y):
  m = y.size
  h = sigmoid(X.dot(theta))
  J = (-1) * (1/m) * (np.log(h).T.dot(y) + np.log(1 - h).T.dot(1 - y)) + (reg/(2*m)) * np.sum(np.square(theta[1:]))
  
  if np.isnan(J[0]): return(np.inf)
  return(J[0])

In [23]:
def lrgradientReg(theta, reg, X, y):
  m = y.size
  h = sigmoid(X.dot(theta.reshape(-1, 1)))
  grad = (1/m) * X.T.dot(h - y) + (reg/m) * np.r_[[[0]], theta[1:].reshape(-1, 1)]
  return(grad.flatten())

One-vs-All Classification

In [27]:
def oneVsAll(features, classes, n_labels, reg):
  initial_theta = np.zeros((features.shape[1], 1))
  all_theta = np.zeros((n_labels, features.shape[1]))

  for c in np.arange(1, n_labels + 1):
    res = minimize(lrcostFunctionReg, initial_theta, args = (reg, features, (classes == c) * 1),
                   method = None, jac = lrgradientReg, options = {"maxiter": 50})
    all_theta[c - 1] = res.x
  return(all_theta)

One-vs-All Prediction

In [30]:
def predictOneVsAll(all_theta, features):
  probs = sigmoid(features.dot(all_theta.T))
  return(np.argmax(probs, axis = 1) + 1)

In [32]:
theta = oneVsAll(X, y, 10, 0.1)
pred = predictOneVsAll(theta, X)
print(f"Training set accuracy: {(np.mean(pred == y.ravel())) * 100}")

Training set accuracy: 93.24


#### Multiclass Logistic Regression with scikit-learn

In [36]:
clf = LogisticRegression(C = 10, solver = "liblinear")
clf.fit(X[:, 1:], y.ravel())

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

확실히 scikit-learn의 Logistic Regression이 penalty나 scaling 등등을 잘 갖춰놔서 일반적(기본 이론적)인 형태보다는 적합이 잘 되는 듯

In [38]:
pred2 = clf.predict(X[:, 1:])
print(f"Training set accuracy: {(np.mean(pred2 == y.ravel())) * 100}")

Training set accuracy: 96.5


#### Neural Networks

In [42]:
def predict(theta1, theta2, features):
  z2 = theta1.dot(features.T)
  a2 = np.c_[np.ones((data['X'].shape[0], 1)), sigmoid(z2).T]
  
  z3 = a2.dot(theta2.T)
  a3 = sigmoid(z3)
  return(np.argmax(a3, axis = 1) + 1)

In [43]:
pred = predict(theta1, theta2, X)
print(f"Training set accuracy: {np.mean(pred == y.ravel()) * 100}")

Training set accuracy: 97.52


그리고 layer가 적긴 하지만, NN으로 들어가니까 확실히 적합률이 높아진다.
물론 지금까지 구해서 확인한 적합률은 모두 training error 측면이니까 test error 측면으로 해석할 수는 없지만, 그럼에도 어떤 모형이 과적합일 수 있냐는 물음엔 NN이 과적합에 가깝다고 답한다.